In [6]:
# adds parent dir to python path
import sys
sys.path.insert(0, '..')

#system lib
import os
import json

# 3rd party lib
import numpy as np
#import matplotlib
#import matplotlib.pyplot as plt
#%matplotlib notebook
#matplotlib.rcParams.update({'font.size': 22})
#from bqplot import pyplot as plt

import matplotlib.pyplot as plt
%matplotlib auto

# astro lib
from astropy.io import fits
import sep

# my lib
import ImageTools as it
import DataTools as dt
from gphelper import GPHelper

from importlib import reload

import warnings
warnings.filterwarnings('ignore')


from multiprocessing import Pool

Using matplotlib backend: Qt5Agg


In [ ]:
def make_nonnegative(img):
    epsilon = np.abs(img.min()) + 1e-3
    img[img<=0] = img[img>0].min()
    return img

def get_random_spheroids(num=10, with_ids=False):
    with open('../spheroids', 'r') as f:
        spheroids = np.array(f.readlines())
    
    if num==-1:
        num = len(spheroids)
    
    selected = np.random.choice(spheroids, num, replace=False)
    
    data_dir = os.path.join(os.getenv('HOME'), 'Documents/astro_data/orig_images')
    fmask = 'GDS_{}_{}.fits'
    f_string = os.path.join(data_dir, fmask)

    sources = []
    for s in selected:
        img = fits.getdata(f_string.format(s.strip(), 'h'))
        segmap = fits.getdata(f_string.format(s.strip(), 'segmap'))
        img_id = int(s.split('_')[1])
        sources.append((img.copy(), segmap.copy(), img_id))
        del img 
        del segmap
    
    if with_ids:
        return list(zip(selected, sources))
    else:
        return sources

# Petrosian Helpers ----------------------------------------------------------
def one_over_eta(rs, fs, R):
    return ((np.pi*R**2) / (np.sum(fs[rs<=R]))) * fs[rs==R]

def petrosian_radius(rs, fs):
    #r_candidates = np.array([one_over_eta(rs, fs, R) for R in rs])
    r_candidates = np.array([fs[rs==r]/fs[rs<=r].mean() for r in rs]).flatten()
    r_candidates = r_candidates[rs<12.5]
    
    return rs[np.square(r_candidates-0.2).argmin()]

def petrosian_flux(rs, fs, R_p):
    return np.sum(fs[rs<=2*R_p])

def R_x(rs, fs, x):
    x /= 100
    sum_ratio = np.cumsum(fs)/np.sum(fs)
    return rs[np.square(sum_ratio-x).argmin()]

def petrosian_Re(R_50, R_90):
    P_3 = 8e-6
    P_4 = 8.47
    R_ratio = min(R_90/R_50, 3.8)
    return R_50 / (1 - (P_3 * (R_ratio)**P_4))

def get_rs_and_fs(img, src_map):
    cx, cy = it.img_center(img, src_map)
    xs, ys = np.meshgrid(np.arange(img.shape[0]), np.arange(img.shape[1]).T)
    rs = np.sqrt(np.square(xs-cx)+np.square(ys-cy))
    
    rs = rs.flatten()
    fs = img.flatten()
    sorted_rs = np.argsort(rs)
    
    rs = rs[sorted_rs]
    fs = fs[sorted_rs]
    
    return rs, fs

def get_re(img, src_map):
    rs, fs = get_rs_and_fs(img, src_map)
    
    R_p = petrosian_radius(rs, fs)
    F_p = petrosian_flux(rs, fs, R_p)
    p_mask = rs <= 2*R_p
    
    _rs = rs[p_mask]
    _fs = fs[p_mask]

    R_50 = R_x(_rs, _fs, 50)
    R_90 = R_x(_rs, _fs, 90)

    re = petrosian_Re(R_50, R_90)
    re_idx = np.square(_rs-re).argmin()
    
    return rs[re_idx], rs, fs

def get_re(rsfs):
    rs, fs = rsfs
    
    R_p = petrosian_radius(rs, fs)
    F_p = petrosian_flux(rs, fs, R_p)
    p_mask = rs <= 2*R_p
    
    _rs = rs[p_mask]
    _fs = fs[p_mask]

    R_50 = R_x(_rs, _fs, 50)
    R_90 = R_x(_rs, _fs, 90)

    re = petrosian_Re(R_50, R_90)
    re_idx = np.square(_rs-re).argmin()
    
    return rs[re_idx]

# Petrosian Helpers ----------------------------------------------------------

def denoise(img, segmap, img_id):
    noise_bank = img[segmap==0].flatten()
    other_source = np.logical_and(segmap!=0, segmap!=img_id)
    np.place(img, other_source, noise_bank)
    
    array_sep_likes = img.byteswap().newbyteorder()
    bkg = sep.Background(img.newbyteorder('='), mask=segmap==img_id, bw=10, bh=10)
    img = img-bkg
    
    return img

# https://ned.ipac.caltech.edu/level5/March05/Graham/Graham2.html
def sersic(x):
    def b(n):
        return 1.9992*n-0.3271

    def I(r):
        """Assuming that I_e=1.0 and that R_e=1.0"""
        n = 4
        return np.exp(-b(n) * (np.power(r, 1/n)  - 1))

    return I(x)

In [ ]:
_id, (img, segmap, img_id) = get_random_spheroids(num=1, with_ids=True)[0]
img = denoise(img, segmap, img_id)
plt.title(_id)
plt.imshow(img, cmap='gray')


In [ ]:
rs, fs = get_rs_and_fs(img, segmap==img_id)

str = 'r, I(r)\n'
for r, f in zip(rs, fs):
    str += f'{r},{f}\n'
    
with open('ascii_rsfs.csv', 'w') as f:
    f.write(str)

In [ ]:
rs, fs = [], []
with open('ascii_rsfs.csv', 'r') as f:
    for line in f:
        if 'I(r)' in line:
            continue
        nums = line.split(' ')
        rs.append(float(nums[0]))
        fs.append(float(nums[1]))
        
rs = np.array(rs)
fs = np.array(fs)

In [ ]:
plt.plot(rs, fs)

In [ ]:
%%time
dR=0.05*(rs.max()-rs.min())
print(dR)
f_smooth = dt.loessc_p(rs,fs,dx=dR, pnum=2)

In [ ]:
plt.plot(rs, fs)
plt.plot(rs, f_smooth)

In [ ]:
(f_smooth<0).sum()

In [ ]:
num_interp = 100
interp_rs = np.linspace(rs.min(), rs.max(), num_interp)
interp_fs = np.interp(interp_rs, rs, f_smooth)
interp_fs[interp_fs<0] = 0

plt.plot(interp_rs, interp_fs)

In [ ]:
L = np.zeros(num_interp)
L[0] = np.pi*interp_rs[0]**2  * interp_fs[0]
for i in range(1,num_interp,1):
    L[i] = L[i-1] + np.pi*(interp_rs[i]**2 - interp_rs[i-1]**2)*interp_fs[i]

In [ ]:
plt.plot(interp_rs, L)

In [ ]:
A = np.pi * interp_rs**2

In [ ]:
etas = interp_fs*A/L
plt.plot(interp_rs, etas)
plt.xlim([0,10])

In [ ]:
eta_idx = np.square(etas[interp_rs<10]-0.2).argmin()
interp_rs[eta_idx]

In [ ]:
etas[eta_idx]

In [7]:
spheroids = get_random_spheroids(num=-1)

In [ ]:
%%time
#_ = process_img(spheroids[6])

#processed = list(map(process_img, spheroids[:32]))

with Pool(4) as p:
    processed = list(p.map(process_img, spheroids[:]))

In [14]:
rps = [p[4] for p in processed]

In [34]:
plt.hist(rps, bins=20, alpha=0.5)

(array([  2.,   1.,   9.,  13.,  30.,  23.,  39.,  30.,  33.,  30.,  15.,
         17.,  19.,  28.,  19.,  15.,  13.,  51.,  35.,  78.]),
 array([ 5.53898668,  5.76202853,  5.98507037,  6.20811221,  6.43115406,
         6.6541959 ,  6.87723774,  7.10027958,  7.32332143,  7.54636327,
         7.76940511,  7.99244695,  8.2154888 ,  8.43853064,  8.66157248,
         8.88461432,  9.10765617,  9.33069801,  9.55373985,  9.77678169,
         9.99982354]),
 <a list of 20 Patch objects>)

In [32]:
res = [p[2] for p in processed]

In [33]:
plt.title('$R_e$')
plt.hist(res, bins=20, alpha=0.5)

(array([  2.,   1.,  12.,  18.,  29.,  36.,  30.,  49.,  31.,  19.,  20.,
         23.,  28.,  21.,  17.,  17.,  45.,  47.,  43.,  12.]),
 array([  6.38189199,   6.69936569,   7.01683939,   7.3343131 ,
          7.6517868 ,   7.9692605 ,   8.2867342 ,   8.60420791,
          8.92168161,   9.23915531,   9.55662901,   9.87410272,
         10.19157642,  10.50905012,  10.82652382,  11.14399753,
         11.46147123,  11.77894493,  12.09641863,  12.41389234,  12.73136604]),
 <a list of 20 Patch objects>)

In [23]:
max_rs = [max(p[0]/p[2]) for p in processed]

In [24]:
plt.hist(max_rs, bins=20, alpha=0.5)

(array([ 46.,  79.,  41.,  23.,  35.,  35.,  20.,  27.,  30.,  40.,  31.,
         22.,  28.,  21.,   8.,   9.,   3.,   0.,   1.,   1.]),
 array([ 4.64169893,  4.8743245 ,  5.10695006,  5.33957563,  5.57220119,
         5.80482676,  6.03745232,  6.27007789,  6.50270345,  6.73532902,
         6.96795458,  7.20058015,  7.43320571,  7.66583128,  7.89845684,
         8.13108241,  8.36370797,  8.59633354,  8.8289591 ,  9.06158467,
         9.29421024]),
 <a list of 20 Patch objects>)

In [18]:
plt.figure()
plt.title("Spheroid SBP")
for (rs, fs, re, ie, R_p, I_p) in processed:
    plt.plot(rs/R_p, np.log10(make_nonnegative(fs/I_p)), color='b', alpha=0.2)
plt.xlabel("R/$R_p$")
plt.ylabel("I/$I_p$ (log)")

In [46]:
rs = [p[0] for p in processed]
fs = [p[1] for p in processed]
ies = [p[3] for p in processed]

sm, lg = [],[]
for re, rp, mr, r, f, ie in zip(res, rps, max_rs, rs, fs, ies):
    if rp<=9.25:
        sm.append((re, rp, mr, r/re, f/ie))
    else:
        lg.append((re, rp, mr, r/re, f/ie))

plt.figure()
plt.title('Small population Petrosian Radius <= 9.25')
plt.hist([s[0] for s in sm], bins=20, label='Re', alpha=0.5)
plt.hist([s[1] for s in sm], bins=20, label='Rp', alpha=0.5)
plt.hist([s[2] for s in sm], bins=20, label='Norm-Length', alpha=0.5)
plt.legend()
    
plt.figure()
plt.title('Small population sbp')
plt.xlim(0,5)
for _, _, _, r, f in sm:
    plt.plot(r,np.log10(f), color='b', alpha=0.2)
    
plt.figure()
plt.title('Large population Petrosian Radius > 9.25')
plt.hist([s[0] for s in lg], bins=20, label='Re', alpha=0.5)
plt.hist([s[1] for s in lg], bins=20, label='Rp', alpha=0.5)
plt.hist([s[2] for s in lg], bins=20, label='Norm-Length', alpha=0.5)
plt.legend()

plt.figure()
plt.title('Large population sbp')
plt.xlim(0,5)
for _, _, _, r, f in lg:
    plt.plot(r,np.log10(f), color='b', alpha=0.2)
        
        

In [180]:
def petrosian_radius(rs, fs):
    #r_candidates = np.array([one_over_eta(rs, fs, R) for R in rs])
    r_candidates = np.array([fs[rs==r]/fs[rs<=r].mean() for r in rs]).flatten()
    r_candidates = r_candidates[rs<12.5]
    
    return rs[np.square(r_candidates-0.2).argmin()]


def make_nonnegative(img):
    epsilon = np.abs(img.min()) + 1e-3
    if img.min() <= 0:
        img += epsilon
    return img

def get_random_spheroids(num=10, with_ids=False):
    with open('../spheroids', 'r') as f:
        spheroids = np.array(f.readlines())
    
    if num==-1:
        num = len(spheroids)
    
    selected = np.random.choice(spheroids, num, replace=False)
    
    data_dir = os.path.join(os.getenv('HOME'), 'Documents/astro_data/orig_images')
    fmask = 'GDS_{}_{}.fits'
    f_string = os.path.join(data_dir, fmask)

    sources = []
    for s in selected:
        img = fits.getdata(f_string.format(s.strip(), 'h'))
        segmap = fits.getdata(f_string.format(s.strip(), 'segmap'))
        img_id = int(s.split('_')[1])
        sources.append((img.copy(), segmap.copy(), img_id))
        del img 
        del segmap
    
    if with_ids:
        return list(zip(selected, sources))
    else:
        return sources

    
from itertools import starmap
def process_img(args):
    def dx_collections(X, Y, dx):
        collections = []

        for x in X:
            dist_x = np.abs(X-x)
            dist_x = dist_x<=dx
            collections.append((x, X[dist_x], Y[dist_x]))

        return collections

    def weight_function(d):
        return (1 - (d/d.max())**3)**3

    def loess_point(x, X, Y):
        if len(Y) == 1:
            return Y

        _Y = weight_function(np.abs(X-x))*Y        
        y = np.poly1d(np.polyfit(X, _Y, 2))(x)

        return y

    def loessc_p(x, y, dx, pnum=1):
        if dx is None:
            dx = 1.0e-3*np.abs(max(x)-min(x))

        collections = dx_collections(x, y, dx)

        if pnum==1:
            y = starmap(loess_point, collections)

        return np.array(list(y)).flatten()
    
    
    
    def petrosian_radius(rs, fs):
        #r_candidates = np.array([one_over_eta(rs, fs, R) for R in rs])
        r_candidates = np.array([fs[rs==r]/fs[rs<=r].mean() for r in rs]).flatten()
        r_candidates = r_candidates[rs<20.0]

        return rs[np.square(r_candidates-0.2).argmin()]
    
    
    def denoise(img, segmap, img_id):
        noise_bank = img[segmap==0].flatten()
        other_source = np.logical_and(segmap!=0, segmap!=img_id)
        np.place(img, other_source, noise_bank)

        sep_arr = img.byteswap().newbyteorder('N')
        #print(sep_arr.dtype.byteorder)
        bkg = sep.Background(sep_arr, mask=segmap==img_id, bw=10, bh=10)
        img = img-bkg

        return img
    
    def get_rs_and_fs(img, src_map):
        cx, cy = it.img_center(img, src_map)
        xs, ys = np.meshgrid(np.arange(img.shape[0]), np.arange(img.shape[1]).T)
        rs = np.sqrt(np.square(xs-cx)+np.square(ys-cy))

        rs = rs.flatten()
        fs = img.flatten()
        sorted_rs = np.argsort(rs)

        rs = rs[sorted_rs]
        fs = fs[sorted_rs]

        return rs, fs
    
    img, segmap, img_id = args
    src_map = segmap==img_id
    img = denoise(img, segmap, img_id)
    
    #plt.figure()
    #plt.imshow(img, cmap='gray')
    
    rs, fs = get_rs_and_fs(img, src_map)
    #pixel_Rp = petrosian_radius(rs, fs)
    
    #plt.figure()
    #plt.plot(rs, fs)
    
    
    dr = 0.05*(rs.max()-rs.min())
    #print(fs[:10])
    fs = loessc_p(rs, fs, dr, pnum=1)
    #print(fs[:10])
    fs[fs<0]=0
    
    #plt.figure()
    #plt.plot(rs, fs)
    
    num_interp = 100
    _rs = np.linspace(rs.min(), rs.max(), num_interp)
    fs = np.interp(_rs, rs, fs)
    rs = _rs
    
    #plt.figure()
    #plt.plot(rs, fs)
    
    L = np.zeros(num_interp)
    L[0] = np.pi*rs[0]**2  * fs[0]
    for i in range(1,num_interp,1):
        L[i] = L[i-1] + np.pi*(rs[i]**2 - rs[i-1]**2)*fs[i]

    A = np.pi * rs**2
    #print(rs, fs)
    etas = fs*A/L
    r_lim = rs<20.0
    mono_dec = np.argmax(np.diff(etas)>=0)
    mono_dec = np.arange(len(etas))<mono_dec
    r_lim = np.logical_and(r_lim, mono_dec)
    
    if np.all(r_lim==False):
        return (None, None, None, None, None, None, None, None)
    
    
    R_p = np.interp(0.2, np.flipud(etas[r_lim]), np.flipud(rs[r_lim]))
    
    #print(etas)
    
    #plt.figure()
    #plt.plot(etas[r_lim], rs[r_lim], '.')
    #plt.vlines([0.2], 0, 12.5)
    #print(np.flipud(etas[r_lim]), rs[r_lim])
    #print(f'R_p:{R_p}')
    
    #plt.figure()
    #plt.plot(rs, etas)
    
    Fp_idx = np.square(rs - 2*R_p).argmin()
    #print(Fp_idx)
    
    if Fp_idx==0:
        return (None, None, None, None, None, None, None, None)
    
    F_p = np.cumsum(L[:Fp_idx]/L[:Fp_idx].sum())
    I_p = np.interp(R_p, rs, fs)
    
    #plt.figure()
    #plt.plot(F_p, rs[:Fp_idx])
    #print(len(F_p), len(rs[:Fp_idx]))
    
    R_50 = np.interp(.5, F_p, rs[:Fp_idx])
    R_90 = np.interp(.9, F_p, rs[:Fp_idx])
    #print(F_p, rs[:Fp_idx])
    #print(f'R_50:{R_50} R_90:{R_90}')
    
    
    P3, P4 = 8e-6, 8.47
    re = R_50 / (1 - P3 * (R_90 / R_50)**P4)
    ie = np.interp(re, rs, fs)
    
    return (rs, fs, re, ie, R_p, I_p, etas, None)

In [59]:
rps = [(p[4], p[7]) for p in processed]
line, pixel = zip(*rps)
plt.title("2d $R_e$ vs 1D $R_e$ - Data")
plt.xlabel("1D $R_e$")
plt.ylabel("2D $R_e$")
plt.scatter(line, pixel)

mx = np.ceil(max(max(line), max(pixel)))
mn = np.floor(min(min(line), min(pixel)))

plt.plot(np.arange(mn, mx), np.arange(mn, mx), color='c')

In [62]:
plt.title('1/$\eta(R)$ and $R/R_p$ - Data')
plt.ylabel('1/$\eta(R)$')
plt.xlabel('$R/R_p$')
for (rs, fs, re, ie, R_p, I_p, etas, pixel_Rp) in processed:
    plt.plot(rs/R_p, etas, color='b', alpha=0.2)

In [50]:
reload(dt)

re_max_ratio = []

res = []
ies = []
normed_rs = []
normed_fs = []
count = 0

small_res = []

for img, segmap, img_id in spheroids:
    count += 1
    
    print(count/len(_spheroids), end='\r')

    #background subtract images
    src_map = segmap==img_id
    
    img = denoise(img, segmap, img_id)
    
    # get the sorted r vals and i vals
    rs, fs = get_rs_and_fs(img, src_map)
    
    # smooth the fs using loess
    #fs = dt.loessc(rs, fs, .60)
    dR = 0.05*(R.max()-R.min())
    fs = dt.loessc_p(rs, fs, dR, pnum=2)
    #print(len(rs), len(fs))
    #fs = lowess(fs, rs)
    #print(len(rs), len(fs))
    
    # measure petrosian re 
    re = get_re((rs,fs))

    if re<3:
        small_res.append((img_id, re))
    
    re_idx = np.where(rs==re)[0][0] 
    
    ie = fs[re_idx]
    
    #sum_to_re = fs[rs<=re_idx].sum()
    #src_L = sum_to_re*2
    #img_L = fs.sum()
    #if src_L>img_L:
    #    re_max_ratio.append(sum_to_re/img_L)
    #    continue
    
    #max_r = np.square(np.cumsum(fs)-src_L).argmin()
    
    #max_r = rs[max_r]
    
    #fs = fs
    #rs = rs
    pos = lambda a: a if (a<=0).sum()==0 else a + ap.abs(a.min()) + 1e-3
    normed_rs.append(pos(rs))
    normed_fs.append(fs)
    ies.append(ie)
    res.append(re)    

NameError: name '_spheroids' is not defined

In [162]:
plt.title('Effective Radius')
plt.xlabel('$R_e$(Pixels)')
plt.hist(res, bins=50)

(array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.  ,  0.02,  0.04,  0.06,  0.08,  0.1 ,  0.12,  0.14,  0.16,
         0.18,  0.2 ,  0.22,  0.24,  0.26,  0.28,  0.3 ,  0.32,  0.34,
         0.36,  0.38,  0.4 ,  0.42,  0.44,  0.46,  0.48,  0.5 ,  0.52,
         0.54,  0.56,  0.58,  0.6 ,  0.62,  0.64,  0.66,  0.68,  0.7 ,
         0.72,  0.74,  0.76,  0.78,  0.8 ,  0.82,  0.84,  0.86,  0.88,
         0.9 ,  0.92,  0.94,  0.96,  0.98,  1.  ]),
 <a list of 50 Patch objects>)

In [121]:
stat_vals.shape

(1000, 500)

In [126]:
interped_marks = np.concatenate([np.linspace(0, 1, 50, endpoint=False), np.linspace(1, 20, 950)])
#stat_vals = np.dstack(stat_vals)[0,...]
f_mean = np.nanmedian(stat_vals, axis=1)
f_16 = np.nanpercentile(stat_vals, 16, axis=1)
f_84 = np.nanpercentile(stat_vals, 84, axis=1)

plt.figure()
plt.plot(interped_marks, f_mean, color='r', label='$median$', zorder=100)
plt.fill_between(interped_marks, f_16, f_84, color='r', alpha=0.45, label="$16^{th}-84^{th}$", zorder=100)
plt.legend()
plt.show()

In [145]:
interped_marks = np.concatenate([np.linspace(0, 1, 50, endpoint=False), np.linspace(1, 20, 950)])

stat_vals = []

plt.figure()
plt.title('Normalized Surface Brightness')
plt.xlabel('$R/R_e$')
plt.ylabel('$I/I_e$ (Log)')

for (rs, fs, _, _, re, ie, _, _) in processed:
    color = 'b'
    if np.sum(fs<=0) > 0:
        fs = fs + np.abs(fs.min()) + 1e-3
        ie = np.interp(re, rs, fs)
    
    r = rs/re
    f = fs/ie
    plt.plot(r, np.log10(f), color='k', alpha=.05)
    stat_vals.append(np.interp(interped_marks, r, f, left=np.nan, right=np.nan))
    

stat_vals = np.log10(np.dstack(stat_vals)[0,...])
f_mean = np.nanmedian(stat_vals, axis=1)
f_16 = np.nanpercentile(stat_vals, 16, axis=1)
f_84 = np.nanpercentile(stat_vals, 84, axis=1)
    

plt.plot(interped_marks, f_mean, color='r', label='$median$', zorder=100)
plt.fill_between(interped_marks, f_16, f_84, color='r', alpha=0.45, label="$16^{th}-84^{th}$", zorder=100)
plt.legend()
plt.show()

In [35]:
with open('source_rs.csv', 'w') as f:
    f.write(','.join([str(v) for v in interped_marks]) + '\n')
    for i in range(stat_vals.shape[1]):
        f.write(','.join([str(v) for v in stat_vals[:,i]]) + '\n')
    f.write('---next is 16th,median,84th\n')
    for a in [f_16,f_mean,f_84]:
        f.write(','.join([str(v) for v in a]) + '\n')    

In [13]:
stat_vals.shape

(1000, 500)

In [127]:
plt.scatter(res, ies)
plt.xlabel('$R_e$')
plt.ylabel('$I_e$')
plt.title('$R_e$ $I_e$ Correlation')
np.corrcoef(res, ies)

array([[ nan,  nan],
       [ nan,  nan]])

In [146]:
valid_points = ~np.isnan(stat_vals).all(axis=1)

X = interped_marks[valid_points]
Y = f_mean[valid_points]
a = f_84[valid_points] - f_mean[valid_points]

vals = {'x':dt._nmpy_encode(X),
        'y':dt._nmpy_encode(Y),
        'a':dt._nmpy_encode(a)}
with open('vals_for_gp.json', 'w') as f:
    json.dump(vals,f)

In [21]:
with open('vals_for_gp.json', 'r') as f:
    vals = json.load(f)
    
X = dt._nmpy_decode(vals['x'])
Y = dt._nmpy_decode(vals['y'])
a = dt._nmpy_decode(vals['a'])

print(X.shape, Y.shape, a.shape)

(464,) (464,) (464,)


In [20]:
upto=5

In [148]:
# truncate lines from 5re and on
upto = 5
_X = X[X<=upto]
_Y = Y[X<=upto]
_a = a[X<=upto]
tmp = _Y-_a


plt.figure()

print(np.isnan(_X).sum(), np.isnan(_Y).sum(), np.isnan(_a).sum())
split_idx = 0#np.argmin(np.diff(_a))
split_val = _X[split_idx]

# fit and pad line to smooth
plt.plot(_X[split_idx:],_Y[split_idx:], label='Measured Values')
plt.fill_between(_X[split_idx:], tmp[split_idx:], _Y[split_idx:]+_a[split_idx:], alpha=0.2)

_X = _X[split_idx:]
_Y = _Y[split_idx:]
_a = _a[split_idx:]

_X, _Y = dt.pad_line(_X, _Y, 10, 10, append=False)
_, _a = dt.pad_line(_X, _a, 10, 10, append=False)

plt.plot(_X[_X<split_val], _Y[_X<split_val], '--', label='Linear Fit Values')
plt.fill_between(_X[_X<split_val], _Y[_X<split_val]-_a[_X<split_val], _Y[_X<split_val]+_a[_X<split_val], alpha=0.2)

plt.xlabel('$R/R_e$')
plt.ylabel('$I/I_e (Log)$')
plt.title('Measured Surface Brightness W/Linear Fit Projection')
plt.legend()

0 0 0


In [149]:
gp = GPHelper()
gp.fit(_X[:,np.newaxis], _Y, _a, optimize='both')
_x, std = gp.predict(_X[:, np.newaxis], return_std=True)

In [133]:
plt.figure()
plt.plot(_X,_Y, label='GP Fit')
plt.fill_between(_X, _Y-std, _Y+std, alpha=0.2)
plt.legend()

In [24]:
gp = GPHelper(restore_file='gp.json')

In [134]:
test_vals = np.concatenate([np.linspace(0.01, 1, 10, endpoint=False), np.linspace(1, upto, upto*10)])

In [150]:
#samples = gp.sample(test_vals[:,np.newaxis], num_samples=500)
samples = []
for i in range(500):
    print(i/500)
    samples.append(gp.sample(test_vals[:,np.newaxis]))

0.0
0.002ng Samples.    
0.004ng Samples..   
0.006ng Samples.    
0.008ng Samples..   
0.01ing Samples...  
0.012ng Samples     
0.014ng Samples     
0.016ng Samples..   
0.018ng Samples...  
0.02ing Samples     
0.022ng Samples.    
0.024ng Samples..   
0.026ng Samples...  
0.028ng Samples     
0.03ing Samples..   
0.032ng Samples...  
0.034ng Samples.    
0.036ng Samples.    
0.038ng Samples.    
0.04ing Samples     
0.042ng Samples...  
0.044ng Samples...  
0.046ng Samples.    
0.048ng Samples..   
0.05ing Samples     
0.052ng Samples..   
0.054ng Samples...  
0.056ng Samples...  
0.058ng Samples     
0.06ing Samples...  
0.062ng Samples..   
0.064ng Samples..   
0.066ng Samples     
0.068ng Samples...  
0.07ing Samples...  
0.072ng Samples...  
0.074ng Samples...  
0.076ng Samples...  
0.078ng Samples     
0.08ing Samples.    
0.082ng Samples     
0.084ng Samples..   
0.086ng Samples..   
0.088ng Samples...  
0.09ing Samples...  
0.092ng Samples...  
0.094ng Samples..   
0.096ng S

0.794ng Samples.    
0.796ng Samples...  
0.798ng Samples...  
0.8wing Samples.    
0.802ng Samples     
0.804ng Samples     
0.806ng Samples.    
0.808ng Samples     
0.81ing Samples...  
0.812ng Samples..   
0.814ng Samples.    
0.816ng Samples...  
0.818ng Samples.    
0.82ing Samples...  
0.822ng Samples..   
0.824ng Samples.    
0.826ng Samples...  
0.828ng Samples..   
0.83ing Samples     
0.832ng Samples...  
0.834ng Samples.    
0.836ng Samples.    
0.838
0.84ing Samples...  
0.842ng Samples..   
0.844ng Samples.    
0.846ng Samples...  
0.848
0.85ing Samples..   
0.852ng Samples.    
0.854ng Samples..   
0.856ng Samples.    
0.858ng Samples..   
0.86ing Samples.    
0.862ng Samples.    
0.864ng Samples..   
0.866ng Samples     
0.868ng Samples     
0.87ing Samples...  
0.872ng Samples..   
0.874ng Samples     
0.876ng Samples..   
0.878ng Samples..   
0.88ing Samples..   
0.882ng Samples..   
0.884ng Samples..   
0.886ng Samples...  
0.888ng Samples     
0.89ing Samples..   
0

In [152]:
samples = np.stack(samples)

In [153]:
samples.shape

(500, 60, 1)

In [37]:
np.percentile(samples, 16, axis=0).shape

(60,)

In [39]:
with open('gp_vals.csv', 'w') as f:
    f.write(','.join([str(v) for v in test_vals]) + '\n')
    
    for i in range(500):
        f.write(','.join([str(v) for v in samples[i,:]]) + '\n')
    
    f.write('---next is 16th,median,84th\n')
    for i in [16, 50, 84]:
        f.write(','.join([str(v) for v in np.percentile(samples, i, axis=0)]))

In [26]:
plt.title('GP 500 Samples')
plt.xlabel('$R/R_e$')
plt.ylabel('$I/I_e$ (Log)')

for i in range(samples.shape[0]):
    plt.plot(test_vals, samples[i,:,0], color='b', alpha=0.2)

AttributeError: 'list' object has no attribute 'shape'

In [171]:
x = test_vals
gp_res = []
etas_rrp = []
rp_comp = []
plt.figure()
plt.title("R vs 1/$\eta(R)$")
plt.xlabel("1/$\eta(R)$")
plt.ylabel("R")
for (_, _, re, _, R_p_i, ie, _, _), fs in zip(processed, list(samples[:,:,0])):
    fs = 10**fs * ie
    rs = x * R_p_i

    L = np.zeros_like(fs)
    L[0] = np.pi*rs[0]**2  * fs[0]
    for i in range(1,len(fs),1):
        L[i] = L[i-1] + np.pi*(rs[i]**2 - rs[i-1]**2)*fs[i]

    A = np.pi * rs**2
    etas = fs*A/L
    r_lim = rs<20.0
    mono_dec = np.argmax(np.diff(etas)>=0)
    mono_dec = np.arange(len(etas))<mono_dec
    r_lim = np.logical_and(r_lim, mono_dec)
    
    R_p = np.interp(0.2, np.flipud(etas[r_lim]), np.flipud(rs[r_lim]))
    plt.plot(etas[r_lim], rs[r_lim])

    Fp_idx = np.square(rs - 2*R_p).argmin()
    F_p = np.cumsum(L[:Fp_idx]/L[:Fp_idx].sum())
    R_50 = np.interp(.5, F_p, rs[:Fp_idx])
    R_90 = np.interp(.9, F_p, rs[:Fp_idx])
    
    
    P3, P4 = 8e-6, 8.47
    gp_re = R_50 / (1 - P3 * (R_90 / R_50)**P4)
    ie = np.interp(re, rs, fs)
    
    gp_res.append((re, gp_re))
    etas_rrp.append((rs/R_p, etas))
    rp_comp.append((R_p_i, R_p))

In [172]:
plt.figure()
plt.title('Data $R_p$ vs Model $R_p$')
plt.xlabel('Data $R_p$')
plt.ylabel('Model $R_p$')

data_rp, model_rp = zip(*rp_comp)
plt.scatter(data_rp, model_rp)
plt.plot()
mx = np.ceil(max(max(r), max(gr)))
mn = np.floor(min(min(r), min(gr)))
plt.plot(np.arange(mn, mx), np.arange(mn, mx), color='c')
plt.legend()

In [114]:
plt.figure()
plt.title('1/$\eta(R)$ and $R/R_p$ - Model')
plt.ylabel('1/$\eta(R)$')
plt.xlabel('$R/R_p$')
for r, f in etas_rrp:
    plt.plot(r, f, color='b', alpha=0.2)

In [165]:
plt.figure()
r, gr = zip(*gp_res)
plt.title('Input $R_e$ vs Measured $R_e$ - GP Generated SBP')
plt.xlabel('Input $R_e$')
plt.ylabel('Measured $R_e$')
plt.scatter(r, gr, color='b')
mx = np.ceil(max(max(r), max(gr)))
mn = np.floor(min(min(r), min(gr)))
plt.plot(np.arange(mn, mx), np.arange(mn, mx), color='c')

In [84]:
print(len(res), len(gp_res))
plt.hist(res, bins=40, alpha=0.5, label='Data')
#plt.hist(new_res, bins=40, alpha=0.5, label='Line Measured')
#plt.figure()
plt.hist(gp_res, bins=40, alpha=0.5, label='GP')
plt.xlabel("Effective Radius")
plt.legend()

500 500


In [75]:
xs, ys = np.meshgrid(np.arange(84), np.arange(84))
cxy = 42
rs = np.sqrt((xs-cxy)**2 + (ys-cxy)**2)

s = samples[5,:]
_, _, re, ie = processed[35]
rs = rs/re
fs = s*ie


fs = np.interp(rs.flatten(), test_vals, fs).reshape([84,84])
plt.imshow(fs, cmap='gray')

In [76]:
img, segmap, img_id = spheroids[1]

noise = img[segmap==0]

frame = np.zeros([84,84])
np.place(frame, np.ones_like(frame), noise)
np.random.shuffle(frame)
plt.imshow(frame, cmap='gray')

In [81]:
plt.imshow(fs+(frame*0.06), cmap='gray')

In [82]:
fits.PrimaryHDU(frame).writeto('noise_bank.fits')
fits.PrimaryHDU(fs).writeto('gp_img.fits')
fits.PrimaryHDU(fs+(frame*0.06)).writeto('gp_with_noise.fits')